In [4]:
import sys
sys.path.append("../")

In [5]:
from core.preqtester import PreqTester
preqtester = PreqTester("../core/vault/courses.json")

/Users/tarosh/projects/better-section-tally/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 47%|████▋     | 2081/4424 [00:00<00:00, 10565.93it/s]

Error parsing prerequisites for BIOL 01445
preqs: , COMP 01112, ENGR 01201, BIOL 01204
input: Comp II ( COMP 01112 , HONR 01112, or ENGR 01201 ), BIOL 01204 , and one 300 or 400 level class in BIOL or MCB are required as prerequisites to BIOL 01445.
output: ( COMP 01112 HONR 01112 or ENGR 01201 ) BIOL 01204 and or or BIOL 01445

Error parsing prerequisites for CS 04113
preqs: , MATH 01122, MATH 01130
input: MATH 01122 with a minimum grade of D- or MATH 01130 with a minimum grade of D- or S02 with a minimum score of 600 or S12 with a minimum score of 620 or A02 with a minimum score of 27 or CLM with a minimum score of 60 or CLMR with a minimum score of 60 or AAF with a minimum score of 276
output: MATH 01122 or MATH 01130 or or or or or or

Error parsing prerequisites for CS 06205
preqs: , CS 04113, CS 04111, CS 04103, MATH 03150
input: CS 04113 with a minimum grade of C- or CS 04111 with a minimum grade of C- or CS 04103 with a minimum grade of C-  and MATH 03160 with a minimum grade o

100%|██████████| 4424/4424 [00:00<00:00, 9759.86it/s] 

Error parsing prerequisites for JRN 02356
preqs: , JRN 02310, JRN 02321
input: JRN 02310 , JRN 02321 , and 60 overall earned credits are required as prerequisites to JRN 02356.
output: JRN 02310 JRN 02321 and JRN 02356

Error parsing prerequisites for MGT 06330
preqs: , ECON 04101, ECON 04102, MGT 06300
input: ECON 04101 , ECON 04102 , and MGT 06300 are required as prerequisites to MGT 06330.
output: ECON 04101 ECON 04102 and MGT 06300 MGT 06330

Error parsing prerequisites for MKT 09200
preqs: , COMP 01111, COMP 01105
input: COMP I ( COMP 01111 , HONR 01111, or COMP 01105 ) and 12 credits overall are required as prerequisites to MKT 09200.
output: ( COMP 01111 HONR 01111 or COMP 01105 ) and MKT 09200

Error parsing prerequisites for MATH 01130
preqs: , MATH 01122, MATH 01124, MATH 01125, MATH 03125, MATH 01125
input: MATH 01122 with a minimum grade of C- or ( MATH 01124 with a minimum grade of C- and MATH 01125 with a minimum grade of C-) or ( MATH 03125 with a minimum grade of C- and

In [7]:
invalid = 0
for c in preqtester.cache.values():
    if c:
        if not c['valid']:
            invalid += 1

print(f'{invalid} of {len(preqtester.cache)} are invalid')

28 of 4423 are invalid


In [8]:
import pandas as pd
courses = pd.read_json("../core/vault/courses.json")
courses.head()

,CourseCode,CourseTitle,Credits,Description,Prerequisites
0,ACC 02314,Individual Taxation,3,ACC 02314 - Individual Taxation Credits: 3 Pre...,", ACC 03311"
1,ACC 03150,Introduction to Business and Analytics for Fin...,3,ACC 03150 - Introduction to Business and Analy...,None
2,ACC 03210,Principles of Accounting I,3,ACC 03210 - Principles of Accounting I Credits...,None
3,ACC 03211,Principles of Accounting II,3,ACC 03211 - Principles of Accounting II Credit...,", ACC 03210"
4,ACC 03300,Supervised Internship in Accounting,3,ACC 03300 - Supervised Internship in Accountin...,", ACC 03310"


In [10]:
def find_course(courses: pd.Series, course: str) -> int:
    return courses[courses['CourseCode'] == course].index[0]

In [11]:
"(CS 06417 or CS 09427 ) and ( CS 01205 or CS 04215 )"

'(CS 06417 or CS 09427 ) and ( CS 01205 or CS 04215 )'

In [16]:
crse = courses.iloc[find_course(courses, "CS 01205")]
print(crse)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [150]:
import re

def extract_desc(desc) -> dict:
    """
    Extracts the prerequisites from the course description
    """
    heading = r"[A-Za-z]+(?: [A-Za-z]+)"
    stop = f"(?=(?:{heading}:)|$)"
    key = f"(?P<key>{heading}):"
    pattern = key + r'\s*(?P<value>.*?)' + stop

    return {m.group("key").strip(): m.group("value").strip() for m in re.finditer(pattern, desc)}

In [151]:
print(crse['Description'])
extract_desc(crse['Description'])

CS 06205 - Computer Organization Credits: 3 This course provides an introduction to computer organization. Students are exposed to the register level architecture of a modern computer and its assembly language. The topics include machine level data representation, von Neumann architecture and instruction execution cycle, memory hierarchy, I/O and interrupts, instruction sets and types, addressing modes, instruction formats and translation. Prerequisite Courses: CS 04113 with a minimum grade of C- OR CS 04111 with a minimum grade of C- OR CS 04103 with a minimum grade of C-  AND MATH 03160 with a minimum grade of C- OR MATH 03150 with a minimum grade of C- ) Course Attributes: CAT, UGRD Academic Department: Computer Science


{'Organization Credits': '3 This course provides an introduction to computer organization. Students are exposed to the register level architecture of a modern computer and its assembly language. The topics include machine level data representation, von Neumann architecture and instruction execution cycle, memory hierarchy, I/O and interrupts, instruction sets and types, addressing modes, instruction formats and translation.',
 'Prerequisite Courses': 'CS 04113 with a minimum grade of C- OR CS 04111 with a minimum grade of C- OR CS 04103 with a minimum grade of C- \xa0AND MATH 03160 with a minimum grade of C- OR MATH 03150 with a minimum grade of C- )',
 'Course Attributes': 'CAT, UGRD',
 'Academic Department': 'Computer Science'}

In [127]:
preqs = re.findall(r"[A-Z]{2,4}\s*\d{4,5}", crse['Description'])
preqs

['PHYS 00210', 'MATH 01122', 'MATH 01130']

In [123]:
list(set(preqs) - set(courses['CourseCode']))

['MK0292']

In [99]:
import re
desc = crse['Description']
print(desc)
re.search(r"(?<=Prerequisite Courses:)(.*)", desc).group(0)

MATH 03125 - Calculus: Techniques and Applications Credits: 3 This course introduces students to the fundamental concepts and techniques of differential and integral calculus.  Emphasis is placed on practical and informative applications of limits, derivatives and integrals in today’s world, with those in business highlighted.  A graphics calculator is required.  Students are expected to have completed an equivalent of the course of College Algebra. Prerequisite Courses: CLM with a minimum score of 060 or CLMR with a minimum score of 060 or MATH 01123 with a minimum grade of C- or MATH 01122 with a minimum grade of C- or S02 with a minimum score of 600 or A02 with a minimum score of 27 or S12 with a minimum score of 620 or MATH 01124 with a minimum grade of C- or AAF with a minimum score of 276 Course Attributes: CAT, GCAT, GNED, QNTL, SM, UGRD Academic Department: Mathematics


' CLM with a minimum score of 060 or CLMR with a minimum score of 060 or MATH 01123 with a minimum grade of C- or MATH 01122 with a minimum grade of C- or S02 with a minimum score of 600 or A02 with a minimum score of 27 or S12 with a minimum score of 620 or MATH 01124 with a minimum grade of C- or AAF with a minimum score of 276 Course Attributes: CAT, GCAT, GNED, QNTL, SM, UGRD Academic Department: Mathematics'

In [100]:
preq_text = crse['Description'].split('Prerequisite Courses:')[1]
preq_text

' CLM with a minimum score of 060 or CLMR with a minimum score of 060 or MATH 01123 with a minimum grade of C- or MATH 01122 with a minimum grade of C- or S02 with a minimum score of 600 or A02 with a minimum score of 27 or S12 with a minimum score of 620 or MATH 01124 with a minimum grade of C- or AAF with a minimum score of 276 Course Attributes: CAT, GCAT, GNED, QNTL, SM, UGRD Academic Department: Mathematics'

In [101]:
import re

course_pattern = r"[A-Z]{2,4} \d{5}"

out = re.findall(course_pattern + "|" + r"\b(?:and|or)|\(|\)", preq_text)
out = " ".join(out).replace("( )", "") # replace empty parentheses

out = re.search(r"\(?\s{,1}" + course_pattern + r"(.*)", out).group(0)
print(out)

 MATH 01123 or MATH 01122 or or or or MATH 01124 or


In [84]:
expr = re.sub(f"({course_pattern})", r"('\1' in taken)", out)
expr.strip()

"( ('COMP 01112' in taken) ('HONR 01112' in taken) or ('ENGR 01201' in taken) ) ('BIOL 01204' in taken) and or or ('BIOL 01445' in taken)"

In [4]:
taken = []
if 'transfer' in transcript1.keys():
    for course in transcript1['transfer']:
        taken.append(course['subject'] + ' ' + course['course_number'])
if 'completed' in transcript1.keys():
    for term in transcript1['completed']:
        for course in term['courses']:
            taken.append(course['subject'] + ' ' + course['course_number'])
taken

['ENGL 02075',
 'FREN 02101',
 'FREN 02102',
 'FREN 02201',
 'INTR 99013',
 'INTR 99013',
 'MATH 01131',
 'PHYS 00210',
 'PHYS 00211',
 'COMP 01111',
 'CS 00100',
 'CS 04113',
 'MATH 01130',
 'MATH 03150',
 'COMP 01112',
 'CS 01395',
 'CS 04114',
 'CS 06205',
 'CS 07210',
 'MUSG 06109',
 'CMS 04206',
 'CS 01205',
 'INTR 01265',
 'MATH 01210',
 'PHYS 00220',
 'CMS 04205',
 'CS 03351',
 'CS 04222',
 'STAT 02290',
 'WA 01302',
 'CS 04222',
 'CS 02480',
 'CS 06395',
 'CS 07340',
 'CS 07459',
 'CS 04315',
 'CS 04321',
 'CS 06420',
 'CS 09410']

,CourseCode,CourseTitle,Credits,Description,Prerequisites
0,ACC 02314,Individual Taxation,3,ACC 02314 - Individual Taxation Credits: 3 Pre...,", ACC 03311"
1,ACC 03150,Introduction to Business and Analytics for Fin...,3,ACC 03150 - Introduction to Business and Analy...,None
2,ACC 03210,Principles of Accounting I,3,ACC 03210 - Principles of Accounting I Credits...,None
3,ACC 03211,Principles of Accounting II,3,ACC 03211 - Principles of Accounting II Credit...,", ACC 03210"
4,ACC 03300,Supervised Internship in Accounting,3,ACC 03300 - Supervised Internship in Accountin...,", ACC 03310"
...,...,...,...,...,...
4419,WA 07391,Writing Fiction,3,WA 07391 - Writing Fiction Credits: 3 This cla...,", WA 07290, WA 07291"
4420,WA 07392,Fundamentals of Playwriting,3,WA 07392 - Fundamentals of Playwriting Credits...,None
4421,WA 07395,Writing Poetry,3,WA 07395 - Writing Poetry Credits: 3 This clas...,", WA 07290"
4422,WA 07396,Worldbuilding,3,WA 07396 - Worldbuilding Credits: 3 In this co...,


In [6]:
courses['Prerequisites'].apply(lambda x: len(x.split(",")) if x is not None else 0).sort_values(ascending=False).head(20)

2006    40
330     20
1209    20
1961    19
3282    17
3266    17
1743    15
2262    14
3361    14
3362    14
3352    13
3357    12
3347    12
1818    12
3331    12
3330    12
3351    11
1519    11
1823    11
3346    11
Name: Prerequisites, dtype: int64

In [29]:
type(courses.iloc[1743])

pandas.core.series.Series

In [9]:
for x in courses['CourseCode']:
    print(x)

ACC 02314
ACC 03150
ACC 03210
ACC 03211
ACC 03300
ACC 03301
ACC 03310
ACC 03311
ACC 03320
ACC 03326
ACC 03405
ACC 03410
ACC 03411
ACC 03415
ACC 03416
ACC 03419
ACC 03425
ACC 03428
ACC 03430
ACC 03431
ACC 03432
ACC 03499
ACC 03500
ACC 03507
ACC 03510
ACC 03511
ACC 03512
ACC 03516
ACC 03599
ACC 98300
ADV 04232
ADV 04330
ADV 04352
ADV 04355
ADV 04360
ADV 04370
ADV 04375
ADV 04405
ADV 04420
ADV 04421
ADV 04432
ADV 04433
ADV 04434
AFST 11104
AFST 11304
AFST 11350
AFST 11450
AFST 53535
ASL 01101
ASL 01102
ASL 01201
ASL 01211
ASL 01301
ASL 01320
ASL 01440
AMST 13101
AMST 13301
AMST 13320
AMST 13402
ANS 00501
ANS 00502
ANS 00503
ANS 00504
ANS 00505
ANS 00506
ANS 00507
ANTH 02200
ANTH 02202
ANTH 02203
ANTH 02205
ANTH 02210
ANTH 02215
ANTH 02221
ANTH 02225
ANTH 02240
ANTH 02245
ANTH 02250
ANTH 02251
ANTH 02270
ANTH 02275
ANTH 02280
ANTH 02290
ANTH 02295
ANTH 02301
ANTH 02305
ANTH 02310
ANTH 02311
ANTH 02312
ANTH 02315
ANTH 02321
ANTH 02322
ANTH 02323
ANTH 02324
ANTH 02326
ANTH 02335
ANTH 02336
A

In [32]:
preqtester._find_possible_combs("CS 04222")

[('MATH 03150', 'CS 04215', 'CS 04114')]